### Data normalization -> cleaning -> merging -> splitting

### Dataset

In [1]:
import pandas as pd
df_turnbackhoax = pd.read_csv("../dataset/dataset-turnbackhoax.csv")
df_antaranews = pd.read_csv("../dataset/dataset-antaranews.csv")
df_detik = pd.read_csv("../dataset/dataset-detik.csv")
df_detik, df_antaranews, df_turnbackhoax

(                                                  Title  \
 0         Bekal "Strategic Foresight" untuk Para Capres   
 1     Mendag Zulhas Jelaskan Pengaruh Geopolitik Ter...   
 2     Kapan Pengumuman TKN Prabowo-Gibran? Ini Jawab...   
 3     Relawan Pengusaha Muda Dorong Ekonomi Indonesi...   
 4     Panda Nababan Sentil Gibran yang Mudah Berkhia...   
 ...                                                 ...   
 2155  Warga Bantargebang Tolak Pembangunan PSEL: Kam...   
 2156  Polda Metro Dalami Foto Pertemuan SYL dan Firl...   
 2157  Gelar World Walking Day, Walkot Tangerang: Mud...   
 2158  Punya Gen Pemicu Alzheimer, Chris Hemsworth Ma...   
 2159  Gebyar Hari Jantung Sedunia, Bupati Mojokerto ...   
 
                                                    Text  \
 0     Pandangan jauh ke depan (foresight) perlu dimi...   
 1     Menteri Perdagangan RI Zulkifli Hasan menjadi ...   
 2     Ketua Harian Partai Gerindra Dasco mengatakan ...   
 3     Relawan Pengusaha Muda Nasional

### Data labeling + cleaning

In [2]:
# data turnbackhoax semuanya berkategori hoax
# data detik dan antara news diambil dari berita dengan tag terverifikasi
# Kategorikan menjadi hoax = 1 dan fakta = 0

df_turnbackhoax['label'] = 1
df_detik['label'] = 0
df_antaranews['label'] = 0

df_detik.rename(columns={'Title': 'judul', 'Text': 'isi', 'Date': 'tanggal', 'Link': 'link'}, inplace=True)
df_combined = pd.concat([df_turnbackhoax[['judul', 'isi', 'tanggal', 'link', 'label']],
                         df_detik[['judul', 'isi', 'tanggal', 'link', 'label']],
                         df_antaranews[['judul', 'isi', 'tanggal', 'link', 'label']]], ignore_index=True)
df_combined

,judul,isi,tanggal,link,label
0,Video Pria Menginjak Al-Qur’an di Klub Malam A...,"Lokasi kejadian bukan di Arab Saudi, tetapi Ma...","November 15, 2024",https://turnbackhoax.id/2024/11/15/salah-video...,1
1,"Tentara AS Mendarat di Israel, Bersiap Lawan H...","Faktanya, konteks asli video adalah dokumentas...","November 15, 2024",https://turnbackhoax.id/2024/11/15/salah-tenta...,1
2,Detik.com Beritakan “Pernyataan Jokowi yang Ti...,TurnBackHoax memeriksa keaslian foto tangkapan...,"November 15, 2024",https://turnbackhoax.id/2024/11/15/salah-detik...,1
3,Budi Arie Dikuliti Prabowo karena Terbukti Bek...,"Faktanya, tidak ada sumber kredibel yang membe...","November 15, 2024",https://turnbackhoax.id/2024/11/15/salah-budi-...,1
4,Anies Bongkar Misi Busuk Prabowo Usai Tom Lemb...,"Faktanya, tidak ada sumber kredibel yang membe...","November 15, 2024",https://turnbackhoax.id/2024/11/15/salah-anies...,1
...,...,...,...,...,...
6115,Kemendagri sebut Otsus Papua tidak sama dengan...,Pengaturan otonomi khusus Provinsi Papua yang ...,7 Oktober 2019,https://www.antaranews.com/berita/1101222/keme...,0
6116,Rekrutmen partai nasional disebut prioritaskan...,Rekrutmen partai politik nasional di Provinsi ...,7 Oktober 2019,https://www.antaranews.com/berita/1100900/rekr...,0
6117,MK gelar sidang uji aturan Otsus Papua,Mahkamah Konstitusi (MK) akan menggelar sidang...,9 September 2019,https://www.antaranews.com/berita/1051760/mk-g...,0
6118,Bacawali Surabaya pertanyakan formulir syarat ...,Bakal Calon Wali Kota Surabaya jalur perseoran...,7 September 2019,https://www.antaranews.com/berita/1049664/baca...,0


In [3]:
df_combined.value_counts('label')

label
0    3060
1    3060
Name: count, dtype: int64

In [4]:
df_combined.isna().sum()

judul      0
isi        0
tanggal    0
link       0
label      0
dtype: int64

In [5]:
# gabung teks judul dan isi
df_combined['berita'] = df_combined['judul']
df_combined['berita'][0]

'Video Pria Menginjak Al-Qur’an di Klub Malam Arab Saudi\t\t\t\t'

In [6]:
# ubah berita menjadi lowercase dan hanya alphanumeric

df_combined['berita'] = df_combined['berita'].str.lower()
df_combined['berita'] = df_combined['berita'].str.replace(r'[^a-z0-9\s]', '')

# hilangkan whitespace berlebihan
df_combined['berita'] = df_combined['berita'].str.split().str.join(' ')
df_combined['berita'][0]

# hapus stopwords
from nltk.corpus import stopwords
stop = set(stopwords.words('indonesian'))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop])

In [7]:
df_clean = df_combined.drop(columns=['judul', 'isi', 'tanggal', 'link'])
df_clean['berita'] = df_clean['berita'].apply(remove_stopwords)
df_clean['berita'][0]

'video pria menginjak al-qur’an klub malam arab saudi'

### Split data untuk training(80%), test(20%)

In [8]:
from sklearn.model_selection import train_test_split
# Split the data into training and remaining data (80% training, 20% remaining)
train_text, remaining_text, train_label, remaining_label = train_test_split(df_clean['berita'], df_clean['label'], test_size=0.3, random_state=42)

# Split the remaining data into test and validation (20% test, 10% validation)
test_text, val_text, test_label, val_label = train_test_split(remaining_text, remaining_label, test_size=0.3333, random_state=42)

# Convert the splits back into DataFrames
train_df = pd.DataFrame({'berita': train_text, 'label': train_label}).reset_index(drop=True)
test_df = pd.DataFrame({'berita': test_text, 'label': test_label}).reset_index(drop=True)
val_df = pd.DataFrame({'berita': val_text, 'label': val_label}).reset_index(drop=True)

train_df.head(), test_df.head()

(                                              berita  label
 0  independensi hakim disorot putusan mk batas us...      0
 1  bantu ekonomi rakyat, raffi ahmad buka situs judi      1
 2  ombudsman panggil bey machmudin insiden relawa...      0
 3            perundungan perbaikan kurikulum merdeka      0
 4  jokowi temui pangeran arab saudi, teken kerja ...      0,
                                               berita  label
 0        anies dukungan ridwan kamil pilkada jakarta      1
 1  coca-cola ulang tahun, bagi-bagi rp10 juta rat...      1
 2                 gebyar undian berhadiah bank jatim      1
 3  akun tiktok prabowo giveaway 1 unit motor temb...      1
 4    partai prima harap kpu kemudahan partai politik      0)

In [9]:
train_df.value_counts('label'), test_df.value_counts('label'), 

(label
 0    2165
 1    2119
 Name: count, dtype: int64,
 label
 1    630
 0    594
 Name: count, dtype: int64)

In [10]:
# train_df.to_csv("../dataset/train_data.csv", index=False)
# val_df.to_csv("../dataset/val_data.csv", index=False)
# test_df.to_csv("../dataset/test_data.csv", index=False)

train_df.to_csv("../dataset/train_data.csv", index=False)
test_df.to_csv("../dataset/test_data.csv", index=False)
val_df.to_csv("../dataset/val_data.csv", index=False)